## Signature Verification using Siamese Networks

In [24]:
import tensorflow as tf
import os

In [25]:
path = '/kaggle/input/signature-verification-dataset/sign_data/train'
ls = os.listdir(path)
print('training data directories',len(ls))

training data directories 128


#### Preprocess & Pair the signatures


input format: ( (img1, img2), label)  
label = 1: Two genuine signatures from the same person.  
label = 0: One genuine and one forged signature from the same person.  

In [26]:
import random
from PIL import Image
import numpy as np

def load_signature_pairs(data_dir):
    pairs = []
    labels = []
    
    users = sorted([name for name in os.listdir(data_dir) if '_' not in name])
    
    for user in users:
        genuine_dir = os.path.join(data_dir, user)
        forg_dir = genuine_dir + '_forg'
        
        genuine_imgs = os.listdir(genuine_dir)
        forgery_imgs = os.listdir(forg_dir)
        
        num_genuine = len(genuine_imgs)
        
        # Random positive pairs (genuine vs different genuine)
        random_indices = [
            random.choice([j for j in range(num_genuine) if j != i])
            for i in range(num_genuine)
        ]
        
        for i in range(num_genuine):
            img1 = os.path.join(genuine_dir, genuine_imgs[i])
            img2 = os.path.join(genuine_dir, genuine_imgs[random_indices[i]])
            pairs.append((img1, img2))
            labels.append(1)

        # Negative pairs (genuine vs forgery)
        for i in range(min(len(genuine_imgs), len(forgery_imgs))):
            img1 = os.path.join(genuine_dir, genuine_imgs[i])
            img2 = os.path.join(forg_dir, forgery_imgs[i])
            pairs.append((img1, img2))
            labels.append(0)
    
    return pairs, labels


In [27]:
path = '/kaggle/input/signature-verification-dataset/sign_data/train'
pairs, labels = load_signature_pairs(path)
print(len(pairs), len(labels))

1606 1606


In [28]:
import os

for p1, p2 in pairs[:5]:
    print(p1, os.path.exists(p1))
    print(p2, os.path.exists(p2))


/kaggle/input/signature-verification-dataset/sign_data/train/001/001_15.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_01.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_18.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_05.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_24.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_22.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_02.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_15.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_08.PNG True
/kaggle/input/signature-verification-dataset/sign_data/train/001/001_05.PNG True


### backbone

caluculate embeddings for each image to compare similarity (how close they are in space)  
Input: 224×224 grayscale image  
Output: 256-dimensional embedding for input image.

In [29]:
import tensorflow as tf
from tensorflow.keras import layers, Model

IMG_SIZE = 224

# ---------------------------
# Custom Layer for L2 Normalization
# ---------------------------
class L2Normalization(layers.Layer):
    def call(self, inputs):
        return tf.nn.l2_normalize(inputs, axis=1)

# ---------------------------
# Backbone Builder
# ---------------------------
def build_backbone():
    """
    Builds a CNN backbone .
    """
    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 1))
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = L2Normalization()(x)  # Normalized embeddings
    return Model(inputs, x, name="SimpleCNNBackbone")


Feature Extraction:
Pass both images through the backbone (shared weights) to get embeddings.  
Similarity Calculation:
Use L1 distance (absolute difference) between the two embeddings.  
Output Layer:
sigmoid activation gives a similarity score (between 0 and 1).  


### siamese neural network

In [30]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# ---------------------------
# Custom Layer for L1 Distance
# ---------------------------
class L1Distance(layers.Layer):
    """Compute element-wise absolute difference between two embeddings."""
    def call(self, inputs):
        return tf.abs(inputs[0] - inputs[1])

# ---------------------------
# Siamese Network Builder
# ---------------------------
def build_siamese_network(backbone):
    input_1 = layers.Input(shape=(224, 224, 1), name="image_1")
    input_2 = layers.Input(shape=(224, 224, 1), name="image_2")

    # Extract embeddings from backbone
    embed_1 = backbone(input_1)
    embed_2 = backbone(input_2)

    # Compute L1 distance using custom layer
    distance = L1Distance()([embed_1, embed_2])

    # Final classification head
    output = layers.Dense(1, activation='sigmoid')(distance)

    # Build the model
    model = Model(inputs=[input_1, input_2], outputs=output, name="SiameseNetwork")
    return model


In [31]:
backbone = build_backbone()
siamese_model = build_siamese_network(backbone)
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
siamese_model.summary()


Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_1             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_2             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ SimpleCNNBackbone   │ (None, 256)       │    107,520 │ image_1[0][0],    │
│ (Functional)        │                   │            │ image_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance_1       │ (None, 256)       │          0 │ SimpleCNNBackbon… │
│ (L1Distance)        │                   │            │ SimpleCNNBackbon… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        257 │ l1_distance_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 107,777 (421.00 KB)

 Trainable params: 107,777 (421.00 KB)

 Non-trainable params: 0 (0.00 B)

input image -> grayscale, [224, 224] px, normalized

In [32]:
def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1)  # grayscale
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image  # shape: (224, 224, 1)

In [33]:
def make_tf_dataset(pairs, labels, batch_size=32, shuffle=True):
    path_ds = tf.data.Dataset.from_tensor_slices((pairs, labels))

    def load_images(pair, label):
        img1 = preprocess_image(pair[0])
        img2 = preprocess_image(pair[1])
        return (img1, img2), label

    dataset = path_ds.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
    # num_parallel_calls=tf.data.AUTOTUNE lets TensorFlow load multiple images in parallel = faster.
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [34]:
train_dataset = make_tf_dataset(pairs, labels, batch_size=32)
print(len(train_dataset))

51


In [35]:
siamese_model.fit(train_dataset, epochs=10)

Epoch 1/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.6851 - loss: 0.6775
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.8537 - loss: 0.5747
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.8834 - loss: 0.5235
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.9062 - loss: 0.4985
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.8961 - loss: 0.4617
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9111 - loss: 0.4672
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9145 - loss: 0.4644
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9041 - loss: 0.4481
Epoch 9/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9211 - loss: 0.4045
Epoch 10/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.9239 - loss: 0.3957


In [36]:
# Load test data
path = '/kaggle/input/signature-verification-dataset/sign_data/test'
pairs, labels = load_signature_pairs(path)

print("Total samples:", len(pairs))

test_dataset = make_tf_dataset(pairs, labels, batch_size=32)
print("Batches:", tf.data.experimental.cardinality(test_dataset).numpy())

# Evaluate model
loss, accuracy = siamese_model.evaluate(test_dataset)
print(f"\nTest Accuracy: {accuracy:.4f}")

Total samples: 476
Batches: 15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.9775 - loss: 0.3317

Test Accuracy: 0.9748


In [37]:
print(test_dataset)


<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [38]:
img_path1 = '/kaggle/input/signature-verification-dataset/sign_data/train/017_forg/01_0107017.PNG'
img_path2 = '/kaggle/input/signature-verification-dataset/sign_data/train/017/01_017.png'
def predict_similarity(model, img_path1, img_path2):
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)

    # Add batch dimension: (1, 224, 224, 1)
    img1 = tf.expand_dims(img1, axis=0)
    img2 = tf.expand_dims(img2, axis=0)

    prediction = model.predict([img1, img2])[0][0]  # sigmoid output

    print(f"Similarity Score: {prediction:.4f}")
    if prediction >= 0.55:
        print("Genuine")
    else:
        print("Forged ")

In [39]:
predict_similarity(siamese_model, img_path1, img_path2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
Similarity Score: 0.0414
Forged 


save the model

In [45]:
siamese_model.save("Siamese_signature.keras")


In [47]:
import tensorflow as tf

siamese_model = tf.keras.models.load_model(
    "Siamese_signature.keras",
    custom_objects={"L1Distance": L1Distance, "L2Normalization": L2Normalization}
)

converter = tf.lite.TFLiteConverter.from_keras_model(siamese_model)
tflite_model = converter.convert()

# حفظه
with open("Siamese_signature.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpgmkmzezo/assets


INFO:tensorflow:Assets written to: /tmp/tmpgmkmzezo/assets


Saved artifact at '/tmp/tmpgmkmzezo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name='image_1'), TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name='image_2')]
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139512553729168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553728592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553729936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553727248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553728400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553726672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553724752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139512553730128: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1764714853.189159     322 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1764714853.189185     322 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


In [52]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import gradio as gr
from PIL import Image
import os
import sys

# Image size used during model training (constant)
IMG_SIZE = 224

# Verification threshold (55%): Defines the minimum required similarity for a match
VERIFICATION_THRESHOLD = 0.55

# 1. Custom Layer Definitions (Mandatory for model loading)
# These definitions are necessary because the trained model uses these custom layers.
class L2Normalization(layers.Layer):
    """Applies L2 normalization to the input vector."""
    def call(self, inputs):
        return tf.nn.l2_normalize(inputs, axis=1)

class L1Distance(layers.Layer):
    """Calculates the absolute difference between two vectors."""
    def call(self, inputs):
        return tf.abs(inputs[0] - inputs[1])

# Required list for loading the model with custom layers
CUSTOM_OBJECTS = {
    "L2Normalization": L2Normalization,
    "L1Distance": L1Distance
}

# 2. Model Path Definition
MODEL_PATH = '/kaggle/working/Siamese_signature.keras'

# 3. Fallback Model Building Function (Used if the trained file is not found)
def build_fallback_model():
    """Builds a simple Siamese model with random weights."""
    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 1), name="input_image")
    # Simple CNN backbone structure
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.GlobalAveragePooling2D()(x)
    embed = layers.Dense(256, activation='relu')(x)
    embed = L2Normalization()(embed)
    
    input_2 = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 1), name="image_2")
    
    # Create the backbone model for re-use
    backbone_model = Model(inputs, embed)
    
    embed_1 = backbone_model(inputs)
    embed_2 = backbone_model(input_2)
    
    distance = L1Distance()([embed_1, embed_2])
    output = layers.Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[inputs, input_2], outputs=output, name="SiameseFallback")
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# 4. Model Loading
try:
    # Attempt to load the trained model
    siamese_model = tf.keras.models.load_model(MODEL_PATH, custom_objects=CUSTOM_OBJECTS)
    print(f"✅ Model loaded successfully from: {MODEL_PATH}. Ready for prediction.")
except Exception as e:
    # Fallback plan if the file is not found
    print(f"❌ Failed to load model from {MODEL_PATH}. Please ensure the saved file is in the notebook path.")
    print(f"Building a fallback model with random weights instead of stopping. Error: {e}")
    siamese_model = build_fallback_model()
    # Exit to prevent use of the untrained model if the trained one is mandatory
    sys.exit("Stopped: Trained model not found. Please ensure the file path is correct.")


# 5. Image Pre-processing Function
def preprocess_image(img_arr):
    """
    Processes the image: converts to grayscale, resizes, and normalizes to fit the model input.
    """
    if img_arr is None:
        return None
    
    # Convert to grayscale if not already
    if img_arr.ndim == 3 and img_arr.shape[-1] == 3:
        img = Image.fromarray(img_arr).convert('L')
        img_arr = np.array(img)
    elif img_arr.ndim == 2:
        pass
    else:
        try:
            img = Image.fromarray(img_arr).convert('L')
            img_arr = np.array(img)
        except:
            return None
        
    # Resize
    img = tf.image.resize(img_arr[..., np.newaxis], [IMG_SIZE, IMG_SIZE])
    
    # Normalization
    img = tf.cast(img, tf.float32) / 255.0
    
    # Add batch dimension
    return tf.expand_dims(img, axis=0)

# 6. Gradio Prediction Function
def predict_match(image1, image2):
    """
    Takes two images, processes them, and uses the model for prediction.
    """
    img1_processed = preprocess_image(image1)
    img2_processed = preprocess_image(image2)

    if img1_processed is None or img2_processed is None:
        # Error message for the user in English
        return gr.Label("Please upload a valid image for both inputs for comparison.", color="yellow", label="Verification Result")

    # Perform prediction using the trained model
    try:
        # Prediction requires a list of inputs: [image1, image2]
        prediction = siamese_model.predict([img1_processed, img2_processed], verbose=0)[0][0]
    except Exception as e:
        # Error message for the user in English
        return gr.Label(f"An error occurred during prediction. Error: {e}", color="red", label="Prediction Error")
    
    # Interpret the result and compare against the threshold (0.55)
    similarity_percentage = prediction * 100
    
    # Check for match based on the threshold
    if prediction >= VERIFICATION_THRESHOLD:
        match_status = "Genuine ✅ (Match)"
        color = "green"
    else:
        match_status = "Forged ❌ (No Match)"
        color = "red"
        
    output_text = match_status
    
    return gr.Label(output_text, color=color, label="Verification Result")

# 7. Create Gradio Interface
iface = gr.Interface(
    fn=predict_match,
    inputs=[
        # Input labels in English
        gr.Image(label="(Image 1)", type="numpy"),
        gr.Image(label="(Image 2)", type="numpy")
    ],
    # Output label in English
    outputs=gr.Label(label="Verification Result"),
    # Title and description in English
    title="Signature Verification ✍️🖋️",
    description=f"Upload two signatures to classify as Genuine ✅ or Forged ❌.",
    allow_flagging="never",
    # Interface theme
    theme=gr.themes.Soft()
)

# 8. Launch the Application
iface.launch(share=True)


✅ Model loaded successfully from: /kaggle/working/Siamese_signature.keras. Ready for prediction.


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://1ec774457269333e0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
